In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 

In [45]:
#function to format data into data points
def makeDataPoints(data):
    newDataPoints=np.empty([len(data)-1,12])
    newTargets=np.empty([len(data)-1,1])
    for g in range(len(data)):
        datapoint = 0
        d0,fp0 =data[g,1], data[g,2]
        if g == 1:
            d1,fp1=data[g-1,1], data[g-1,2]
            datapoint=np.array([g,d0,d1,fp1,d1,fp1,d1,fp1,d1,fp1,d1,fp1])
        if g == 2:
            d1,fp1=data[g-1,1], data[g-1,2]
            d2,fp2=data[g-2,1], data[g-2,2]
            datapoint=np.array([g,d0,d1,fp1,d1,fp1,d1,fp1,d1,fp1,d2,fp2])
        if g == 3:
            d1,fp1=data[g-1,1], data[g-1,2]
            d2,fp2=data[g-2,1], data[g-2,2]
            d3,fp3=data[g-3,1], data[g-3,2]
            datapoint=np.array([g,d0,d1,fp1,d1,fp1,d1,fp1,d2,fp2,d3,fp3])
        if g == 4:
            d1,fp1=data[g-1,1], data[g-1,2]
            d2,fp2=data[g-2,1], data[g-2,2]
            d3,fp3=data[g-3,1], data[g-3,2]
            d4,fp4=data[g-4,1], data[g-4,2]
            datapoint=np.array([g,d0,d1,fp1,d1,fp1,d2,fp2,d3,fp3,d4,fp4])
        if g >= 5:
            d1,fp1=data[g-1,1], data[g-1,2]
            d2,fp2=data[g-2,1], data[g-2,2]
            d3,fp3=data[g-3,1], data[g-3,2]
            d4,fp4=data[g-4,1], data[g-4,2]
            d5,fp5=data[g-5,1], data[g-5,2]
            datapoint=np.array([g,d0,d1,fp1,d2,fp2,d3,fp3,d4,fp4,d5,fp5])
        newTargets[g-1]=fp0
        newDataPoints[g-1]=datapoint
    return newDataPoints, newTargets

In [46]:
#Making data into useable numpy arrays to be passed into the final dataset creation filter
data = pd.read_csv(r'C:\Users\Parker Hill\Desktop\ECE 196\NFL Running Back\FinalData.csv') #Change to directory that your data is in
data = pd.DataFrame(data)
length = len(data['Week'])
previousweeknum = -1
array = np.empty([1,3])
dataPoints = np.empty([1,12])
targets = np.empty([1,1])
for i in range(0,length):
    weeknum = data['Week'][i]
    if weeknum < previousweeknum:
        array = np.delete(array,(0),axis=0) #I dunno how to make a truly empty array, so need to delete the first row each time
        #This line you would call your make data points function on with the created array, I used a print statement for verification
        newDataPoints,newTargets = makeDataPoints(array)
        dataPoints = np.vstack([dataPoints,newDataPoints])
        targets = np.vstack([targets,newTargets])
        #print(array)
        array = np.empty([1,3])
    rank = data['Rank'][i]
    fantasy = data['Fantasy'][i]
    newrow = np.array([weeknum, rank, fantasy])
    array = np.vstack([array, newrow])
    previousweeknum = weeknum
dataPoints = np.delete(dataPoints,(0),axis=0)
targets = np.delete(targets,(0),axis=0)
print(dataPoints)
dataPoints.shape

[[ 1.   7.  31.  ... 23.1 31.  23.1]
 [ 2.  14.   7.  ... 25.4 31.  23.1]
 [ 3.  28.  14.  ... 25.4 31.  23.1]
 ...
 [ 8.   3.  11.  ...  5.  15.  10.9]
 [ 9.   6.   3.  ...  7.9 20.   5. ]
 [10.   1.   6.  ... 11.4 32.   7.9]]


(5224, 12)

In [57]:
traindata_x = dataPoints[0:4000]
traindata_y = targets[0:4000]
testdata_x = dataPoints[4001:5224]
testdata_y = targets[4001:5224]

[[ 1.   7.  31.  ... 23.1 31.  23.1]
 [ 2.  14.   7.  ... 25.4 31.  23.1]
 [ 3.  28.  14.  ... 25.4 31.  23.1]
 ...
 [ 4.   8.  31.  ...  4.1 15.   3.1]
 [ 5.  10.   8.  ...  4.1 15.   3.1]
 [ 6.   6.  10.  ... 21.4  9.   4.1]]
[[25.4]
 [23.4]
 [16.6]
 ...
 [15.8]
 [ 9.4]
 [ 9.2]]


In [1]:
# Adapted from https://www.pluralsight.com/guides/predictive-analytics-with-pytorch
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [58]:
class ANN(nn.Module):
    def __init__(self):
        input_dim = 12
        output_dim = 1
        super(ANN,self).__init__()
        self.fc1 = nn.Linear(input_dim,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,32)
        self.output_layer = nn.Linear(32,output_dim)
        self.dropout = nn.Dropout(0.15)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)
        return nn.Sigmoid()(x)

In [59]:
model = ANN()
print(model)

ANN(
  (fc1): Linear(in_features=12, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
)


In [64]:
X_train = torch.from_numpy(traindata_x)
Y_train = torch.from_numpy(traindata_y)
X_test = torch.from_numpy(testdata_x)
Y_test = torch.from_numpy(testdata_y)

torch.Size([4000, 1])


In [67]:
train = torch.utils.data.TensorDataset(X_train,Y_train)
test = torch.utils.data.TensorDataset(X_test,Y_test)
train_loader = torch.utils.data.DataLoader(train,batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test,batch_size = 64, shuffle = True)


In [70]:
import torch.optim as optim
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, weight_decay = 1e-6, momentum = 0.8)

In [75]:
epochs = 2000
epoch_list = []
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []

# lines 7 onwards
model.train() # prepare model for training

for epoch in range(epochs):
    train_loss = 0.0
    valloss = 0.0
    
    correct = 0
    total = 0
    for data,target in train_loader:
        data = Variable(data).float()
        target = Variable(target).type(torch.FloatTensor)
        optimizer.zero_grad()
        output = model(data)
        predicted = (torch.round(output.data[0]))
        total += len(target)
        correct += (predicted == target).sum()

        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        trainloss += loss.item()*data.size(0)

    trainloss = trainloss/len(train_loader.dataset)
    accuracy = 100 * correct / float(total)
    train_acc_list.append(accuracy)
    train_loss_list.append(train_loss)
    print('Epoch: {} \tTraining Loss: {:.4f}\t Acc: {:.2f}%'.format(
        epoch+1, 
        train_loss,
        accuracy
        ))
    epoch_list.append(epoch + 1)

Epoch: 1 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 2 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 3 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 4 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 5 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 6 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 7 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 8 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 9 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 10 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 11 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 12 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 13 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 14 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 15 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 16 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 17 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 18 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 19 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 20 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 21 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 22 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 23 	Training

Epoch: 181 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 182 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 183 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 184 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 185 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 186 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 187 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 188 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 189 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 190 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 191 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 192 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 193 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 194 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 195 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 196 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 197 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 198 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 199 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 200 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 201 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 202 	Training Loss: 0.0000	

Epoch: 359 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 360 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 361 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 362 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 363 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 364 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 365 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 366 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 367 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 368 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 369 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 370 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 371 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 372 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 373 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 374 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 375 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 376 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 377 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 378 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 379 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 380 	Training Loss: 0.0000	

Epoch: 537 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 538 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 539 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 540 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 541 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 542 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 543 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 544 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 545 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 546 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 547 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 548 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 549 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 550 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 551 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 552 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 553 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 554 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 555 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 556 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 557 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 558 	Training Loss: 0.0000	

Epoch: 715 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 716 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 717 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 718 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 719 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 720 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 721 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 722 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 723 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 724 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 725 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 726 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 727 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 728 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 729 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 730 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 731 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 732 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 733 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 734 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 735 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 736 	Training Loss: 0.0000	

...

Epoch: 1243 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1244 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1245 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1246 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1247 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1248 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1249 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1250 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1251 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1252 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1253 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1254 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1255 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1256 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1257 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1258 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1259 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1260 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1261 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1262 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1263 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1264 	

Epoch: 1417 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1418 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1419 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1420 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1421 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1422 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1423 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1424 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1425 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1426 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1427 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1428 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1429 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1430 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1431 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1432 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1433 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1434 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1435 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1436 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1437 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1438 	

Epoch: 1591 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1592 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1593 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1594 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1595 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1596 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1597 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1598 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1599 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1600 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1601 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1602 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1603 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1604 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1605 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1606 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1607 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1608 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1609 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1610 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1611 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1612 	

Epoch: 1765 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1766 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1767 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1768 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1769 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1770 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1771 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1772 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1773 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1774 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1775 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1776 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1777 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1778 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1779 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1780 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1781 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1782 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1783 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1784 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1785 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1786 	

Epoch: 1939 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1940 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1941 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1942 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1943 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1944 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1945 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1946 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1947 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1948 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1949 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1950 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1951 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1952 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1953 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1954 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1955 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1956 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1957 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1958 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1959 	Training Loss: 0.0000	 Acc: 0.22%
Epoch: 1960 	

In [79]:
correct = 0
total = 0
valloss = 0
model.eval() 

with torch.no_grad():
    for data, target in test_loader:
        data = Variable(data).float()
        target = Variable(target).type(torch.FloatTensor)

        output = model(data)
        loss = loss_fn(output, target)
        valloss += loss.item()*data.size(0)
        
        predicted = (torch.round(output.data[0]))
        total += len(target)
        correct += (predicted == target).sum()
    
    valloss = valloss/len(test_loader.dataset)
    accuracy = 100 * correct/ float(total)
    print(accuracy) 

tensor(0.4088)
